In [3]:
from datasets import Dataset, DatasetDict
import yaml
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
contexts = []
# with open("domain_contexts_eng.jsonl", 'r', encoding='utf-8') as file:
with open("quest_contexts_eng.jsonl", 'r', encoding='utf-8') as file:
    for line in file:
        contexts.append(json.loads(line))

# score_df = pd.read_csv("../data/gpt_results/domain_scores_df.csv")
answer_df = pd.read_csv("eng_kdqol.csv")

In [5]:
new_contexts = [c['context'].split("\n\n\n")[0] for c in contexts]

In [6]:
# query_to_retriever = [
#     "How high did the patient rate their Physical Component Summary score?",
#     "How high did the patient rate their Mental Component Summary score?",
#     "How high did the patient rate their Symptoms and Problems score?",
#     "How high did the patient rate their Effect of Kidney Disease score?",
#     "How high did the patient rate their Burden of Kidney Disease score?"
# ] # domain
query_to_retriever = [
    "Tell me the score of how the patient described their health.",
    "Tell me the score of whether the patient said they were unable to do their work as carefully as usual due to psychological problems in the past 4 weeks.",
    "Tell me the score of whether the patient said they had problems with social activities with family, friends, and neighbors due to physical or psychological health problems in the past 4 weeks.",
    "Tell me the score of whether the patient said they feel they get sick more easily than others.",
    "Tell me the score of whether the patient said they feel frustrated when managing their kidney disease.",
    "Tell me the score of whether the patient said they had trouble recognizing people or had a poor sense of time and place in the past 4 weeks.",
    "Tell me the score of whether the patient said they were somewhat troubled by stress or worry due to kidney disease.",
    "Tell me the score of whether the patient said they were able to sleep as much as needed.",
    "Tell me the score of whether the patient thinks it will be difficult to continue working at their job due to their health.",
    "Tell me the score of whether the patient thinks the doctors and nurses at the dialysis center help them cope well with their kidney disease.",
]

In [7]:
qa_pairs = []
query_num = ["1", "5c", "6", "11a", "12c", "13f", "15f", "18b", "21", "24b"]
for i, (question, num) in enumerate(zip(query_to_retriever, query_num)):
    # Extend the qa_pairs list with new entries for each context-answer pair
    for j, context in enumerate(new_contexts):
        answers = {}
        answer = answer_df.loc[j, num]
        quest_start = context.index(num)
        answer_start = context[quest_start:].index("Answer")+8
        answers["text"] = [str(answer)]
        answers["answer_start"] = [answer_start]
        answers["answer_end"] = [answer_start + len(str(answer))]
        qa_pairs.append([j*10+i, num, context, question, answers])

In [8]:
answer_list = [qa_pair[4]['text'][0] for qa_pair in qa_pairs]

In [9]:
set(answer_list)

{'A good bit of the time',
 'A little of the time',
 'All of the time',
 'Definitely false',
 'Definitely true',
 "Don't know",
 'Excellent',
 'Extremely',
 'Extremely bothered',
 'Fair',
 'Good',
 'Moderately',
 'Moderately bothered',
 'Most of the time',
 'Mostly false',
 'Mostly true',
 'No',
 'None of the time',
 'Not at all',
 'Not at all bothered',
 'Poor',
 'Quite a bit',
 'Slightly',
 'Some of the time',
 'Somewhat bothered',
 'Very good',
 'Very much bothered',
 'Yes',
 'nan'}

In [10]:
qa_df = pd.DataFrame(qa_pairs, columns=["id", "title", "context", "question", "answers"])
train_df, test_df = train_test_split(qa_df, test_size=0.1, stratify=qa_df["title"])

train_dataset = Dataset.from_dict(train_df)
test_dataset = Dataset.from_dict(test_df)

# Create a DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

# Save the dataset_dict to disk
dataset_dict.save_to_disk('./data/quest_qa_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/5085 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/565 [00:00<?, ? examples/s]

In [113]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)

In [54]:
from datasets import load_from_disk, load_dataset
kdqol_dataset = load_from_disk('./data/quest_qa_dataset')
squad_dataset = load_dataset('squad_v2')

In [55]:
print(squad_dataset['validation']['question'][3])

Who was the Norse leader?


In [20]:
print(squad_dataset['validation']['context'][3])

Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.


In [22]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokens = tokenizer(squad_dataset['validation']['context'][3])

/home/changuk/.conda/envs/belt/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [27]:
squad_dataset['validation']['context'][3].index("Denver Broncos")

177

In [26]:
len(squad_dataset['validation']['context'][3])

775

In [25]:
len(tokens['input_ids'])

159

In [ ]:
import numpy as np
np.mean([len(squad_dataset['validation']['answers'][i]['text']) for i in range(len(squad_dataset['validation']['answers']))])

In [56]:
len(squad_dataset['train']['answers'][3]['text'])

1

In [59]:
print(squad_dataset['validation']['answers'][9])

{'text': ['William the Conqueror', 'William the Conqueror', 'William the Conqueror'], 'answer_start': [1022, 1022, 1022]}
